In [1]:
import time
import pandas as pd
import urllib.request
import requests
import bs4
from collections import Counter
import konlpy
from konlpy.tag import Hannanum
import matplotlib.pyplot as plt
from wordcloud import WordCloud

In [2]:
data = pd.read_excel('D:/marketing/data1.xlsx')
a = list(data['실검'])
b = []
c = []
d = []
for i, v in enumerate(a):
    b.append([v,i])
    if i % 7 == 6:
        c.append(max(b)[1])
        b = []
for i in c:
    d.append(str(data.loc[i, '날짜'])[:10].replace('-',''))

In [3]:
e = []
for n, y in enumerate(d):
    e.append((n,y))
e

[(0, '20210225'),
 (1, '20210304'),
 (2, '20210312'),
 (3, '20210323'),
 (4, '20210325'),
 (5, '20210407'),
 (6, '20210412'),
 (7, '20210417'),
 (8, '20210426'),
 (9, '20210505'),
 (10, '20210506'),
 (11, '20210513'),
 (12, '20210524'),
 (13, '20210530'),
 (14, '20210607'),
 (15, '20210610'),
 (16, '20210618'),
 (17, '20210628'),
 (18, '20210703'),
 (19, '20210709'),
 (20, '20210716'),
 (21, '20210725'),
 (22, '20210802'),
 (23, '20210808'),
 (24, '20210812'),
 (25, '20210823'),
 (26, '20210826'),
 (27, '20210902'),
 (28, '20210911'),
 (29, '20210916'),
 (30, '20210923'),
 (31, '20211005'),
 (32, '20211009'),
 (33, '20211018'),
 (34, '20211025'),
 (35, '20211028'),
 (36, '20211105'),
 (37, '20211111'),
 (38, '20211123'),
 (39, '20211129'),
 (40, '20211204'),
 (41, '20211214'),
 (42, '20211216'),
 (43, '20211226'),
 (44, '20220101'),
 (45, '20220109'),
 (46, '20220116'),
 (47, '20220126'),
 (48, '20220201'),
 (49, '20220207'),
 (50, '20220211'),
 (51, '20220222'),
 (52, '20220224'),
 (5

In [4]:
len(e)

115

In [5]:
tagger=Hannanum()
check = ['경향신문','국민일보','동아일보','문화일보','서울신문','세계일보','조선일보','중앙일보','한겨레','한국일보','뉴스1','뉴시스','연합뉴스','채널A','한국경제TV','JTBC','KBS','MBC','MBN','SBS','TV조선','YTN','노컷뉴스','데일리안','미디어오늘','아이뉴스24','오마이뉴스','프레시안','디지털데일리','디지털타임스','블로터','전자신문','ZDNet Korea']
stopwords = ['단독','논란','구독','뉴스','채널','네이버','연합뉴스','금지','재배포','오후','오전','이후','이날','사진','콘텐츠','영상편집','영상취재','기자','만원','때문']

In [ ]:
for n, y in e:
    print("n =", n, "y =", y)
    url = "https://news.naver.com/main/ranking/popularDay.naver?date=" + y
    html = urllib.request.urlopen(url)
    time.sleep(10)

    bs_obj = bs4.BeautifulSoup(html, "html.parser")
    div = bs_obj.findAll("div", {"class":"rankingnews_box"})

    k=1
    data=[]
    col = ['No','Publisher','Rank','Title','URL','Contents']

    for i in div:
        publisher = i.find("strong", {"class":"rankingnews_name"})
        if publisher.text in check:
            ul = i.find("ul", {"class":"rankingnews_list"})
            lis = ul.findAll("li")
            for j in lis:
                em_tag = j.find("em")
                a_tag = j.find("a")

                if (a_tag != None) and (em_tag != None):
                    article_url = a_tag['href']
                    news=urllib.request.urlopen(article_url)
                    news_obj = bs4.BeautifulSoup(news, "html.parser")
                    body = news_obj.find("div", {"id":"newsct_article","class":"newsct_article _article_body"})
                    if body == None:
                        body = news_obj.find("div", {"id":"newsEndContents","class":"news_end"})
                        print("newsEnd")
                        if body == None:
                            print('ERROR')
                            continue
                    contents = body.text.replace('\n','')
                    row = [k, publisher.text, em_tag.text, a_tag.text, article_url, contents]
                    data.append(row)
                    k += 1
            time.sleep(1)
        time.sleep(1)

    df = pd.DataFrame(data, columns = col)
    if len(df) == 0:
        print("ERROR")
    else:
        print("Done")
    time.sleep(10)

    text = df[df['Publisher'].isin(check)]['Contents']
    text_filtered = text.str.replace("[^ㄱ-ㅎ ㅏ-ㅣ 가-힣]"," ")

    #time.sleep(5)

    tokenized_data_filtered=[]
    for j in text_filtered:
        temp = tagger.nouns(j)
        temp = [word for word in temp if len(word)>1]  ## 두글자 이상이면 포함, 리스트 함축
        temp = [word for word in temp if word not in stopwords]
        tokenized_data_filtered.append(temp)

    #time.sleep(5)

    tokens=[]
    for k in tokenized_data_filtered:
        tokens.extend(k)

    #time.sleep(5)

    num_top_tokens=100
    counted_tokens=Counter(tokens)
    top_keywords=dict(counted_tokens.most_common(num_top_tokens))

    #time.sleep(5)

    globals()['tk_df{}'.format(n)] = pd.DataFrame(list(top_keywords.items()), columns=['keywords','freq'])
    globals()['tk_df{}'.format(n)].to_csv('D:/marketing/data5/tk_df' + str(n) + '.csv', encoding = 'utf-8-sig')
    time.sleep(600)
    print('='*10)

n = 0 y = 20210225
newsEnd
Done


C:\Users\Public\Documents\ESTsoft\CreatorTemp\ipykernel_14328\1573155859.py:49: FutureWarning: The default value of regex will change from True to False in a future version.
  text_filtered = text.str.replace("[^ㄱ-ㅎ ㅏ-ㅣ 가-힣]"," ")


n = 1 y = 20210304
newsEnd
Done


C:\Users\Public\Documents\ESTsoft\CreatorTemp\ipykernel_14328\1573155859.py:49: FutureWarning: The default value of regex will change from True to False in a future version.
  text_filtered = text.str.replace("[^ㄱ-ㅎ ㅏ-ㅣ 가-힣]"," ")


n = 2 y = 20210312
newsEnd
Done


C:\Users\Public\Documents\ESTsoft\CreatorTemp\ipykernel_14328\1573155859.py:49: FutureWarning: The default value of regex will change from True to False in a future version.
  text_filtered = text.str.replace("[^ㄱ-ㅎ ㅏ-ㅣ 가-힣]"," ")


In [12]:
for n, y in e[21:]:
    print("n =", n, "y =", y)
    url = "https://news.naver.com/main/ranking/popularDay.naver?date=" + y
    html = urllib.request.urlopen(url)
    time.sleep(10)

    bs_obj = bs4.BeautifulSoup(html, "html.parser")
    div = bs_obj.findAll("div", {"class":"rankingnews_box"})

    k=1
    data=[]
    col = ['No','Publisher','Rank','Title','URL','Contents']

    for i in div:
        publisher = i.find("strong", {"class":"rankingnews_name"})
        if publisher.text in check:
            ul = i.find("ul", {"class":"rankingnews_list"})
            lis = ul.findAll("li")
            for j in lis:
                em_tag = j.find("em")
                a_tag = j.find("a")

                if (a_tag != None) and (em_tag != None):
                    article_url = a_tag['href']
                    news=urllib.request.urlopen(article_url)
                    news_obj = bs4.BeautifulSoup(news, "html.parser")
                    body = news_obj.find("div", {"id":"newsct_article","class":"newsct_article _article_body"})
                    if body == None:
                        body = news_obj.find("div", {"id":"newsEndContents","class":"news_end"})
                        print("newsEnd")
                        if body == None:
                            print('ERROR1')
                            continue
                    contents = body.text.replace('\n','')
                    row = [k, publisher.text, em_tag.text, a_tag.text, article_url, contents]
                    data.append(row)
                    k += 1
            time.sleep(1)
        time.sleep(1)

    df = pd.DataFrame(data, columns = col)
    df.to_csv('D:/marketing/data5/df' + str(n) + '.csv', encoding = 'utf-8-sig')
    if len(df) == 0:
        print("ERROR2")
    else:
        print("Done")
    time.sleep(10)

    text = df[df['Publisher'].isin(check)]['Contents']
    text_filtered = text.str.replace("[^ㄱ-ㅎ ㅏ-ㅣ 가-힣]"," ")

    time.sleep(5)

    tokenized_data_filtered=[]
    for j in text_filtered:
        temp = tagger.nouns(j)
        temp = [word for word in temp if len(word)>1]  ## 두글자 이상이면 포함, 리스트 함축
        temp = [word for word in temp if word not in stopwords]
        tokenized_data_filtered.append(temp)

    time.sleep(5)

    tokens=[]
    for k in tokenized_data_filtered:
        tokens.extend(k)

    time.sleep(5)

    num_top_tokens=100
    counted_tokens=Counter(tokens)
    top_keywords=dict(counted_tokens.most_common(num_top_tokens))

    time.sleep(5)

    tk_df = pd.DataFrame(list(top_keywords.items()), columns=['keywords','freq'])
    tk_df.to_csv('D:/marketing/data5/tk_df' + str(n) + '.csv', encoding = 'utf-8-sig')
    print("sleep 10min")
    time.sleep(600)
    print('#'*30)

n = 14 y = 20210607
Done


C:\Users\Public\Documents\ESTsoft\CreatorTemp\ipykernel_20784\2886238681.py:50: FutureWarning: The default value of regex will change from True to False in a future version.
  text_filtered = text.str.replace("[^ㄱ-ㅎ ㅏ-ㅣ 가-힣]"," ")


sleep 10min
##############################
n = 15 y = 20210610
Done


C:\Users\Public\Documents\ESTsoft\CreatorTemp\ipykernel_20784\2886238681.py:50: FutureWarning: The default value of regex will change from True to False in a future version.
  text_filtered = text.str.replace("[^ㄱ-ㅎ ㅏ-ㅣ 가-힣]"," ")


sleep 10min
##############################
n = 16 y = 20210618
Done


...

C:\Users\Public\Documents\ESTsoft\CreatorTemp\ipykernel_20784\2886238681.py:50: FutureWarning: The default value of regex will change from True to False in a future version.
  text_filtered = text.str.replace("[^ㄱ-ㅎ ㅏ-ㅣ 가-힣]"," ")


sleep 10min
##############################
n = 20 y = 20210716
Done


C:\Users\Public\Documents\ESTsoft\CreatorTemp\ipykernel_20784\2886238681.py:50: FutureWarning: The default value of regex will change from True to False in a future version.
  text_filtered = text.str.replace("[^ㄱ-ㅎ ㅏ-ㅣ 가-힣]"," ")


sleep 10min
##############################
n = 21 y = 20210725


HTTPError: HTTP Error 500: Internal Server Error

In [7]:
len(e)

115

In [9]:
f = []
for n, y in e:
    if n in [2, 6]:
        f.append((n, y))
print(f)

[(2, '20210312'), (6, '20210412')]


In [6]:
import random

In [9]:
for n, y in e[51:]:
    print("n =", n, "y =", y)
    url = "https://news.naver.com/main/ranking/popularDay.naver?date=" + y
    headers = {'User-agent' : 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/113.0.0.0 Safari/537.36'}
    html = requests.get(url, headers = headers)
    time.sleep(10)

    bs_obj = bs4.BeautifulSoup(html.content, "html.parser")
    div = bs_obj.findAll("div", {"class":"rankingnews_box"})

    k=1
    data=[]
    col = ['No','Publisher','Rank','Title','URL','Contents']

    for i in div:
        publisher = i.find("strong", {"class":"rankingnews_name"})
        if publisher.text in check:
            ul = i.find("ul", {"class":"rankingnews_list"})
            lis = ul.findAll("li")
            for j in lis:
                em_tag = j.find("em")
                a_tag = j.find("a")

                if (a_tag != None) and (em_tag != None):
                    article_url = a_tag['href']
#                    news=urllib.request.urlopen(article_url)
#                    news_obj = bs4.BeautifulSoup(news, "html.parser")
#                     body = news_obj.find("div", {"id":"newsct_article","class":"newsct_article _article_body"})
#                     if body == None:
#                         body = news_obj.find("div", {"id":"newsEndContents","class":"news_end"})
#                         print("newsEnd")
#                         if body == None:
#                             print('ERROR1')
#                             continue
#                     contents = body.text.replace('\n','')
                     row = [k, publisher.text, em_tag.text, a_tag.text, article_url, contents]
#                     data.append(row)
#                     k += 1
#             time.sleep(random.randint(1, 4))
#         time.sleep(random.randint(1, 4))

    df = pd.DataFrame(data, columns = col)
    df.to_csv('D:/marketing/data5/df' + str(n) + '.csv', encoding = 'utf-8-sig')
    if len(df) == 0:
        print("ERROR2")
    else:
        print("Done")
    time.sleep(10)

    text = df[df['Publisher'].isin(check)]['Contents']
    text_filtered = text.str.replace("[^ㄱ-ㅎ ㅏ-ㅣ 가-힣]"," ")

    #time.sleep(5)

    tokenized_data_filtered=[]
    for j in text_filtered:
        temp = tagger.nouns(j)
        temp = [word for word in temp if len(word)>1]  ## 두글자 이상이면 포함, 리스트 함축
        temp = [word for word in temp if word not in stopwords]
        tokenized_data_filtered.append(temp)

    #time.sleep(5)

    tokens=[]
    for k in tokenized_data_filtered:
        tokens.extend(k)

    #time.sleep(5)

    num_top_tokens=100
    counted_tokens=Counter(tokens)
    top_keywords=dict(counted_tokens.most_common(num_top_tokens))

    #time.sleep(5)

    tk_df = pd.DataFrame(list(top_keywords.items()), columns=['keywords','freq'])
    tk_df.to_csv('D:/marketing/data5/tk_df' + str(n) + '.csv', encoding = 'utf-8-sig')
    print("sleep 10min")
    time.sleep(600)
    t = random.randint(30, 60)
    time.sleep(t)
    print('#'*30)

n = 51 y = 20220222
Done


C:\Users\Public\Documents\ESTsoft\CreatorTemp\ipykernel_19984\2511201532.py:51: FutureWarning: The default value of regex will change from True to False in a future version.
  text_filtered = text.str.replace("[^ㄱ-ㅎ ㅏ-ㅣ 가-힣]"," ")


sleep 10min
##############################
n = 52 y = 20220224
ERROR2


C:\Users\Public\Documents\ESTsoft\CreatorTemp\ipykernel_19984\2511201532.py:51: FutureWarning: The default value of regex will change from True to False in a future version.
  text_filtered = text.str.replace("[^ㄱ-ㅎ ㅏ-ㅣ 가-힣]"," ")


sleep 10min
##############################
n = 53 y = 20220307
ERROR2


...

C:\Users\Public\Documents\ESTsoft\CreatorTemp\ipykernel_19984\2511201532.py:51: FutureWarning: The default value of regex will change from True to False in a future version.
  text_filtered = text.str.replace("[^ㄱ-ㅎ ㅏ-ㅣ 가-힣]"," ")


sleep 10min
##############################
n = 61 y = 20220502
ERROR2


C:\Users\Public\Documents\ESTsoft\CreatorTemp\ipykernel_19984\2511201532.py:51: FutureWarning: The default value of regex will change from True to False in a future version.
  text_filtered = text.str.replace("[^ㄱ-ㅎ ㅏ-ㅣ 가-힣]"," ")


sleep 10min
##############################
n = 62 y = 20220507


HTTPError: HTTP Error 500: Internal Server Error

In [11]:
for n, y in e:
    print("n =", n, "y =", y)
    url = "https://news.naver.com/main/ranking/popularDay.naver?date=" + y
    headers = {'User-agent' : 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/113.0.0.0 Safari/537.36'}
    html = requests.get(url, headers = headers, timeout = 5)
    #time.sleep(5)

    bs_obj = bs4.BeautifulSoup(html.content, "html.parser")
    div = bs_obj.findAll("div", {"class":"rankingnews_box"})

    k=1
    data=[]
    col = ['No','Publisher','Rank','Title','URL']

    for i in div:
        publisher = i.find("strong", {"class":"rankingnews_name"})
        if publisher.text in check:
            ul = i.find("ul", {"class":"rankingnews_list"})
            lis = ul.findAll("li")
            for j in lis:
                em_tag = j.find("em")
                a_tag = j.find("a")

                if (a_tag != None) and (em_tag != None):
                    article_url = a_tag['href']
                    row = [k, publisher.text, em_tag.text, a_tag.text, article_url]
                    data.append(row)
                    k += 1
#                    news=urllib.request.urlopen(article_url)
#                    news_obj = bs4.BeautifulSoup(news, "html.parser")
#                     body = news_obj.find("div", {"id":"newsct_article","class":"newsct_article _article_body"})
#                     if body == None:
#                         body = news_obj.find("div", {"id":"newsEndContents","class":"news_end"})
#                         print("newsEnd")
#                         if body == None:
#                             print('ERROR1')
#                             continue
#                     contents = body.text.replace('\n','')
#             time.sleep(random.randint(1, 4))
#         time.sleep(random.randint(1, 4))

    df = pd.DataFrame(data, columns = col)
    df.to_csv('D:/marketing/data7/df' + str(n) + '.csv', encoding = 'utf-8-sig')
    if len(df) == 0:
        print("ERROR2")
        error.append(0)
    else:
        print("Done")
        error.append(1)
#     time.sleep(10)

#     text = df[df['Publisher'].isin(check)]['Contents']
#     text_filtered = text.str.replace("[^ㄱ-ㅎ ㅏ-ㅣ 가-힣]"," ")

#     #time.sleep(5)

#     tokenized_data_filtered=[]
#     for j in text_filtered:
#         temp = tagger.nouns(j)
#         temp = [word for word in temp if len(word)>1]  ## 두글자 이상이면 포함, 리스트 함축
#         temp = [word for word in temp if word not in stopwords]
#         tokenized_data_filtered.append(temp)

#     #time.sleep(5)

#     tokens=[]
#     for k in tokenized_data_filtered:
#         tokens.extend(k)

#     #time.sleep(5)

#     num_top_tokens=100
#     counted_tokens=Counter(tokens)
#     top_keywords=dict(counted_tokens.most_common(num_top_tokens))

#     #time.sleep(5)

#     tk_df = pd.DataFrame(list(top_keywords.items()), columns=['keywords','freq'])
#     tk_df.to_csv('D:/marketing/data5/tk_df' + str(n) + '.csv', encoding = 'utf-8-sig')
#     print("sleep 10min")
#     time.sleep(600)
#     t = random.randint(30, 60)
#     time.sleep(t)
#     print('#'*30)

n = 0 y = 20210225
Done
n = 1 y = 20210304
Done
n = 2 y = 20210312
Done
n = 3 y = 20210323
Done
n = 4 y = 20210325
Done
n = 5 y = 20210407
Done
n = 6 y = 20210412
Done
n = 7 y = 20210417
Done
n = 8 y = 20210426
Done
n = 9 y = 20210505
Done
n = 10 y = 20210506
Done
n = 11 y = 20210513
ERROR2
n = 12 y = 20210524
Done
n = 13 y = 20210530
Done
n = 14 y = 20210607
Done
n = 15 y = 20210610
Done
n = 16 y = 20210618
Done
n = 17 y = 20210628
Done
n = 18 y = 20210703
Done
n = 19 y = 20210709
Done
n = 20 y = 20210716
Done
n = 21 y = 20210725
Done
n = 22 y = 20210802
Done
n = 23 y = 20210808
Done
n = 24 y = 20210812
ERROR2
n = 25 y = 20210823
Done
n = 26 y = 20210826
Done
n = 27 y = 20210902
ERROR2
n = 28 y = 20210911
Done
n = 29 y = 20210916
Done
n = 30 y = 20210923
Done
n = 31 y = 20211005
Done
n = 32 y = 20211009
Done
n = 33 y = 20211018
Done
n = 34 y = 20211025
Done
n = 35 y = 20211028
Done
n = 36 y = 20211105
Done
n = 37 y = 20211111
ERROR2
n = 38 y = 20211123
Done
n = 39 y = 20211129
Done
n 

In [10]:
error = []

In [14]:
for idx, i in enumerate(error):
    if i == 0:
        print(idx)

11
24
27
37
52
53
56
57
63
66
67
68
69
70
73
75
77
78
82
83
85
86
88
90
93
95
105
106
